# Spatial join

[Spatial join](http://wiki.gis.com/wiki/index.php/Spatial_Join) is
yet another classic GIS problem. Getting attributes from one layer and
transferring them into another layer based on their spatial relationship
is something you most likely need to do on a regular basis.

The previous materials focused on learning how to perform a `Point in Polygon query <Lesson3-point-in-polygon.html#how-to-check-if-point-is-inside-a-polygon>`__.
We could now apply those techniques and create our own function to perform a spatial join between two layers based on their
spatial relationship. We could for example join the attributes of a polygon layer into a point layer where each point would get the
attributes of a polygon that ``contains`` the point.

Luckily, [spatial join is already implemented in Geopandas](http://geopandas.org/mergingdata.html#spatial-joins)
, thus we do not need to create it ourselves. There are three possible types of
join that can be applied in spatial join that are determined with ``op`` -parameter in the ``gpd.sjoin()`` -function:

-  ``"intersects"``
-  ``"within"``
-  ``"contains"``

Sounds familiar? Yep, all of those spatial relationships were discussed
in the `previous materials <Lesson3-point-in-polygon.html>`__, thus you should know how they work.

Let's perform a spatial join between these two layers:
- **Addresses:** the address-point Shapefile that we `created <Lesson3-table-join.html>`__ and then `reprojected <Lesson3-projections.html>`__ 
- **Population grid:** a Polygon layer that is a 250m x 250m grid showing the amount of people living in the Helsinki Region.
    - The population grid a dataset is produced by the **Helsinki Region Environmental
Services Authority (HSY)** (see [this page](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) to access data from different years).
    - For this lesson we will use the population grid for year 2015, which can be dowloaded as a shapefile [from this link](https://www.hsy.fi/sites/AvoinData/AvoinData/SYT/Tietoyhteistyoyksikko/Shape%20(Esri)/V%C3%A4est%C3%B6tietoruudukko/Vaestotietoruudukko_2015.zip) in the  [Helsinki Region Infroshare
(HRI) open data portal](http://www.hri.fi/en/dataset/vaestotietoruudukko) 


## Download and clean the data

**Execute the following steps in a terminal window**

- Navigate to the data folder

```
    cd data
```

- Download the population grid using wget:

```
    wget https://www.hsy.fi/sites/AvoinData/AvoinData/SYT/Tietoyhteistyoyksikko/Shape%20(Esri)/V%C3%A4est%C3%B6tietoruudukko/Vaestotietoruudukko_2015.zip

```

-  Unzip the file in Terminal into a folder called Pop15 (using -d flag)

```
    $ unzip Vaestotietoruudukko_2015.zip -d Pop15
    $ ls Pop15
    Vaestotietoruudukko_2015.dbf  Vaestotietoruudukko_2015.shp
    Vaestotietoruudukko_2015.prj  Vaestotietoruudukko_2015.shx
```

You should now have a folder ``/data/Pop15`` with files listed
above.

-  Let's read the data into memory and see what we have.


In [3]:
import geopandas as gpd

# Filepath
fp = "data/Pop15/Vaestotietoruudukko_2015.shp"

# Read the data
pop = gpd.read_file(fp)

# See the first rows
pop.head()


,INDEX,ASUKKAITA,ASVALJYYS,IKA0_9,IKA10_19,IKA20_29,IKA30_39,IKA40_49,IKA50_59,IKA60_69,IKA70_79,IKA_YLI80,geometry
0,688,8,31.0,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.99532626 6689749.005069185,..."
1,703,6,42.0,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.99532626 6685998.998064222,..."
2,710,8,44.0,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.99532626 6684249.004130407,..."
3,711,7,64.0,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.99532626 6683999.004997005,..."
4,715,19,23.0,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.99532626 6682998.998461431,..."


Okey so we have multiple columns in the dataset but the most important
one here is the column ``ASUKKAITA`` (*population in Finnish*) that
tells the amount of inhabitants living under that polygon.

-  Let's change the name of that columns into ``pop15`` so that it is
   more intuitive. Changing column names is easy in Pandas / Geopandas
   using a function called ``rename()`` where we pass a dictionary to a
   parameter ``columns={'oldname': 'newname'}``.

In [4]:
# Change the name of a column
pop = pop.rename(columns={'ASUKKAITA': 'pop15'})

# See the column names and confirm that we now have a column called 'pop15'
pop.columns

Index(['INDEX', 'pop15', 'ASVALJYYS', 'IKA0_9', 'IKA10_19', 'IKA20_29',
       'IKA30_39', 'IKA40_49', 'IKA50_59', 'IKA60_69', 'IKA70_79', 'IKA_YLI80',
       'geometry'],
      dtype='object')

-  Let's also get rid of all unnecessary columns by selecting only
   columns that we need i.e. ``pop15`` and ``geometry``

In [5]:
# Columns that will be sected
selected_cols = ['pop15', 'geometry']

# Select those columns
pop = pop[selected_cols]

# Let's see the last 2 rows
pop.tail(2)

,pop15,geometry
5782,9,"POLYGON ((25513499.99632164 6685498.999797418,..."
5783,30244,"POLYGON ((25513999.999929 6659998.998172711, 2..."


Now we have cleaned the data and have only those columns that we need
for our analysis.

## Join the layers

Now we are ready to perform the spatial join between the two layers that
we have. The aim here is to get information about **how many people live
in a polygon that contains an individual address-point** . Thus, we want
to join attributes from the population layer we just modified into the
addresses point layer ``addresses_epsg3879.shp``.

-  Read the addresses layer into memory

In [1]:
# Addresses filpath
addr_fp = r"data/addresses_epsg3879.shp"

# Read data
addresses = gpd.read_file(addr_fp)

# Check the head of the file
addresses.head(2)

NameError: name 'gpd' is not defined

-  Let's make sure that the coordinate reference system of the layers
are identical

In [2]:
# Check the crs of address points
addresses.crs

# Check the crs of population layer
pop.crs

# Do they match? - We can test that
addresses.crs == pop.crs

NameError: name 'addresses' is not defined

Indeed they are identical. Thus, we can be sure that when doing spatial
queries between layers the locations match and we get the right results
e.g. from the spatial join that we are conducting here.

-  Let's now join the attributes from ``pop`` GeoDataFrame into
   ``addresses`` GeoDataFrame by using ``gpd.sjoin()`` -function

In [ ]:
# Make a spatial join
join = gpd.sjoin(addresses, pop, how="inner", op="within")

# Let's check the result
join.head()


Awesome! Now we have performed a successful spatial join where we got
two new columns into our ``join`` GeoDataFrame, i.e. ``index_right``
that tells the index of the matching polygon in the ``pop`` layer and
``pop15`` which is the population in the cell where the address-point is
located.

-  Let's save this layer into a new Shapefile

In [ ]:
# Output path
outfp = r"/home/geo/addresses_pop15_epsg3979.shp"

# Save to disk
join.to_file(outfp)

Do the results make sense? Let's evaluate this a bit by plotting the
points where color intensity indicates the population numbers.

-  Plot the points and use the ``pop15`` column to indicate the color.
   ``cmap`` -parameter tells to use a sequential colormap for the
   values, ``markersize`` adjusts the size of a point, ``scheme`` parameter can be used to adjust the classification method based on `pysal <http://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html>`_, and ``legend`` tells that we want to have a legend.


In [ ]:
import matplotlib.pyplot as plt

# Plot the points with population info
join.plot(column='pop15', cmap="Reds", markersize=7, scheme='natural_breaks', legend=True);

# Add title
plt.title("Amount of inhabitants living close the the point");

# Remove white space around the figure
@savefig population_points.png width=7in
plt.tight_layout()

By knowing approximately how population is distributed in Helsinki, it
seems that the results do make sense as the points with highest
population are located in the south where the city center of Helsinki
is.